<a href="https://colab.research.google.com/github/nbadino/ItaOil/blob/main/oil.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Prima bozza del paper di policy evaluation sui prezzi del carburante dopo che i prezzi medi verranno esposti al di fuori di ogni distributore

In [ ]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import requests
from zipfile import ZipFile, BadZipFile
import xml.etree.ElementTree as ET
from io import BytesIO
!pip install geopy reverse_geocoder
import reverse_geocoder as rg

In [ ]:
def download_and_extract_data():
    # Check if file exists
    if not os.path.isfile("/content/main.zip"):
        print("File not found, downloading...")
        # Execute wget command
        try:
            subprocess.check_call(['wget', 'https://github.com/nbadino/ItaOil/archive/refs/heads/main.zip', '-P', '/content'])
            print("File downloaded, unzipping...")
            subprocess.check_call(['unzip', "/content/main.zip", '-d', '/content'])
        except subprocess.CalledProcessError as e:
            print("Could not download or unzip file: ", e)
    else:
        print("File exists.")


def load_data_from_dir(price_filepath, gas_station_filepath):
    price_files = [f for f in os.listdir(price_filepath) if f.endswith('.csv')]
    gas_station_files = [f for f in os.listdir(gas_station_filepath) if f.endswith('.csv')]

    price_data = pd.concat([
        pd.read_csv(os.path.join(price_filepath, f), sep=';', skiprows=1, error_bad_lines=False)
        .assign(date=datetime.datetime.strptime(f[6:10], '%m%d').replace(year=2023))
        for f in price_files
    ])

    gas_station_data = pd.concat([
        pd.read_csv(os.path.join(gas_station_filepath, f), sep=';', skiprows=1, error_bad_lines=False)
        .assign(date=datetime.datetime.strptime(f[5:9], '%m%d').replace(year=2023))
        for f in gas_station_files
    ])
    data = pd.merge(price_data, gas_station_data, on='idImpianto')
    return data


def plot_italian_data(data):
    benzina_self_0 = data[(data['descCarburante'] == 'Benzina') & (data['isSelf'] == 0)]
    benzina_self_1 = data[(data['descCarburante'] == 'Benzina') & (data['isSelf'] == 1)]

    benzina_self_0_grouped = benzina_self_0.groupby(benzina_self_0['date_x'].dt.date)['prezzo'].mean()
    benzina_self_1_grouped = benzina_self_1.groupby(benzina_self_1['date_x'].dt.date)['prezzo'].mean()

    plt.figure(figsize=(10, 6))
    plt.plot(benzina_self_0_grouped.index, benzina_self_0_grouped.values, label='Benzina, isSelf = 0')
    plt.plot(benzina_self_1_grouped.index, benzina_self_1_grouped.values, label='Benzina, isSelf = 1')
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title('Average Price Over Time For Benzina (isSelf = 0 and 1)')
    plt.legend()
    plt.show()


def clean_coordinates(data):
    data = data.dropna(subset=['Latitudine', 'Longitudine'])
    valid_data = ((data['Latitudine'] >= -90) & (data['Latitudine'] <= 90) &
                  (data['Longitudine'] >= -180) & (data['Longitudine'] <= 180))
    return data[valid_data]


def add_region_info(data):
    data['Latitudine'] = data['Latitudine'].astype(float)
    data['Longitudine'] = data['Longitudine'].astype(float)
    coordinates = list(zip(data['Latitudine'], data['Longitudine']))
    results = rg.search(coordinates)
    data['region'] = [r['admin1'] for r in results]
    return data


def fetch_french_data(fuel_types):
    # Calculate days since 24th July 2023
    start_date = datetime.datetime(2023, 7, 24)
    days_since_start = (datetime.datetime.now() - start_date).days

    # Generate list of dates from the start date to today
    dates = [(start_date + datetime.timedelta(days=i)).strftime('%Y%m%d') for i in range(days_since_start)]

    data_list = []

    for date in dates:
        url = f'https://donnees.roulez-eco.fr/opendata/jour/{date}'
        response = requests.get(url)
        if response.status_code == 200:
            try:
                with ZipFile(BytesIO(response.content)) as zf:
                    file_name = zf.namelist()[0]
                    with zf.open(file_name) as f:
                        tree = ET.parse(f)
                        root = tree.getroot()
                        for pdv in root.findall('pdv'):
                            id_ = pdv.get('id')
                            latitude = pdv.get('latitude')
                            longitude = pdv.get('longitude')
                            cp = pdv.get('cp')
                            pop = pdv.get('pop')
                            for prix in pdv.findall('prix'):
                                fuel_type = prix.get('nom')
                                price = prix.get('valeur')
                                update_time = prix.get('maj')
                                if fuel_type in fuel_types:
                                    data_list.append({
                                        'id': id_, 'latitude': latitude, 'longitude': longitude, 'cp': cp, 'pop': pop,
                                        'fuel_type': fuel_type, 'price': price, 'update_time': update_time, 'date': date
                                    })
            except BadZipFile:
                print(f"The file for date {date} is not a valid zip file.")
        else:
            print(f"No data available for date {date}.")

    df = pd.DataFrame(data_list)
    df['price'] = df['price'].astype(float)
    df['date'] = pd.to_datetime(df['date'])
    df = df.groupby(['date', 'fuel_type'])['price'].mean().unstack('fuel_type')
    return df


def plot_french_data(df):
    df.plot()
    plt.xlabel('Date')
    plt.ylabel('Average Price')
    plt.title('Average Oil Prices Over Time at a National Level')
    plt.legend(title='Fuel Type')
    plt.show()


def plot_comparison(italian_data, french_data_grouped_by_date, italian_fuel_types=['Benzina', 'Gasolio'], french_fuel_types=['Gazole', 'SP95']):
    # Filter the Italian data to include only the selected fuel types
    filtered_data = italian_data[italian_data['descCarburante'].isin(italian_fuel_types)]

    # Get unique regions
    regions = filtered_data['region'].unique()

    # Create a figure with one subplot for each region
    fig, axs = plt.subplots(len(regions), 1, figsize=(10, 6*len(regions)))
    if len(regions) == 1:
        axs = [axs]

    # Plot the data for each region
    for i, region in enumerate(regions):
        for fuel_type in italian_fuel_types:
            # Subset the data for the current region and fuel type
            data = filtered_data[(filtered_data['region'] == region) & (filtered_data['descCarburante'] == fuel_type)]

            # Calculate the mean price by date
            data_grouped_by_date = data.groupby('date_x')['prezzo'].mean()  # Using 'date_x' here

            # Plot the Italian data
            axs[i].plot(data_grouped_by_date.index, data_grouped_by_date.values, label=f'Italy {fuel_type}')

        for fuel_type in french_fuel_types:
            # Plot the French data
            axs[i].plot(french_data_grouped_by_date.index, french_data_grouped_by_date[fuel_type].values, label=f'France {fuel_type}')

        axs[i].set_xlabel('Date')
        axs[i].set_ylabel('Average Price')
        axs[i].set_title(f'Average Oil Prices Over Time in {region}')
        axs[i].legend()

    plt.tight_layout()
    plt.show()
def generate_combined_dataset(italian_data, french_data_grouped_by_date, output_filepath, italian_fuel_types=['Benzina', 'Gasolio'], french_fuel_types=['Gazole', 'SP95']):
    # Filter the Italian data to include only the selected fuel types
    filtered_data = italian_data[italian_data['descCarburante'].isin(italian_fuel_types)]

    # Group Italian data by date and fuel type
    italian_data_grouped = filtered_data.groupby(['date_x', 'descCarburante'])['prezzo'].mean().unstack('descCarburante')
    italian_data_grouped.columns = [f"Italy_{col}" for col in italian_data_grouped.columns]

    # Combine Italian and French data
    combined_data = pd.concat([french_data_grouped_by_date, italian_data_grouped], axis=1)
    combined_data.reset_index(inplace=True)
    combined_data.rename(columns={'index': 'date'}, inplace=True)

    # Generate the 'treated' column
    combined_data['treated'] = (combined_data['date'] >= '2023-08-01').astype(int)

    # Save to CSV
    combined_data.to_csv(output_filepath, index=False)

def generate_combined_dataset_by_region(italian_data, french_data_grouped_by_date, output_folder, italian_fuel_types=['Benzina', 'Gasolio'], french_fuel_types=['Gazole', 'SP95']):
    # Filter the Italian data to include only the selected fuel types
    filtered_data = italian_data[italian_data['descCarburante'].isin(italian_fuel_types)]

    # Get unique regions
    regions = filtered_data['region'].unique()

    # Ensure the output folder exists
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Process data for each region
    for region in regions:
        region_data = filtered_data[filtered_data['region'] == region]
        region_data_grouped = region_data.groupby(['date_x', 'descCarburante'])['prezzo'].mean().unstack('descCarburante')
        region_data_grouped.columns = [f"Italy_{col}" for col in region_data_grouped.columns]

        # Combine Italian and French data
        combined_data = pd.concat([french_data_grouped_by_date, region_data_grouped], axis=1)
        combined_data.reset_index(inplace=True)
        combined_data.rename(columns={'index': 'date'}, inplace=True)

        # Generate the 'treated' column
        combined_data['treated'] = (combined_data['date'] >= '2023-08-01').astype(int)

        # Save to CSV
        region_output_filepath = os.path.join(output_folder, f"combined_data_{region}.csv")
        combined_data.to_csv(region_output_filepath, index=False)
        print(f"Saved data for region {region} to {region_output_filepath}")

In [ ]:
if __name__ == "__main__":
    download_and_extract_data()
    os.chdir("/content/ItaOil-main")
    price_filepath = '/content/ItaOil-main/data/nuovi/prices'
    gas_station_filepath = '/content/ItaOil-main/data/nuovi/gas_station_info'
    merged_data = load_data_from_dir(price_filepath, gas_station_filepath)
    #plot_italian_data(merged_data)
    merged_data = clean_coordinates(merged_data)
    merged_data = add_region_info(merged_data)
    fuel_types = ['Gazole', 'SP95']
    french_data = fetch_french_data(fuel_types)
    #plot_french_data(french_data)
    italian_fuel_types = ['Benzina', 'Gasolio']
    #plot_comparison(merged_data, french_data)
    generate_combined_dataset(merged_data, french_data, 'combined_data.csv')
    generate_combined_dataset_by_region(merged_data, french_data, 'combined_data_by_region')

File exists.


<ipython-input-2-6ee9001a7628>:21: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv(os.path.join(price_filepath, f), sep=';', skiprows=1, error_bad_lines=False)
<ipython-input-2-6ee9001a7628>:21: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv(os.path.join(price_filepath, f), sep=';', skiprows=1, error_bad_lines=False)
<ipython-input-2-6ee9001a7628>:21: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv(os.path.join(price_filepath, f), sep=';', skiprows=1, error_bad_lines=False)
<ipython-input-2-6ee9001a7628>:21: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  pd.read_csv(os.path.join(pric

Loading formatted geocoded file...
Saved data for region Emilia-Romagna to combined_data_by_region/combined_data_Emilia-Romagna.csv
Saved data for region Piedmont to combined_data_by_region/combined_data_Piedmont.csv
Saved data for region Tuscany to combined_data_by_region/combined_data_Tuscany.csv
Saved data for region Lombardy to combined_data_by_region/combined_data_Lombardy.csv
Saved data for region Liguria to combined_data_by_region/combined_data_Liguria.csv
Saved data for region Latium to combined_data_by_region/combined_data_Latium.csv
Saved data for region Apulia to combined_data_by_region/combined_data_Apulia.csv
Saved data for region Campania to combined_data_by_region/combined_data_Campania.csv
Saved data for region Abruzzo to combined_data_by_region/combined_data_Abruzzo.csv
Saved data for region Calabria to combined_data_by_region/combined_data_Calabria.csv
Saved data for region Veneto to combined_data_by_region/combined_data_Veneto.csv
Saved data for region Trentino-Alto 